In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from datetime import timedelta
from notebook_utils import NotebookUtilities
from pandas import DataFrame
import humanize
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape) # (842663, 112)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(829277, 113)



# Replace the tool applied sender missing patient ID

In [18]:

match_series = (frvrs_logs_df.tool_applied_sender == 'Needle_Decomp(Clone) (UnityEngine.GameObject)') & frvrs_logs_df.tool_applied_patient_id.isnull()
df = frvrs_logs_df[match_series]
print(df.shape)
df.sample(6).dropna(axis='columns', how='all').T

(634, 113)


,30595,798067,800075,209546,767808,699930
action_type,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED
action_tick,202220,120083,433125,638934,301693,543277
event_time,2023-03-14 11:03:29,2023-04-21 12:27:00,2023-04-21 10:13:00,2023-07-27 10:45:19,2023-05-10 11:45:00,2023-03-07 13:38:00
session_uuid,a275e363-d113-4f29-873c-a312f732ae0d,3bac8365-232a-4be0-a551-c5214b9f0706,cf953248-e6cf-4544-95ce-4843d0f56f10,e1a418bf-114d-4b8e-b0a1-0a0ff144c0aa,b0a0d36e-4fb8-43b7-a966-d730e1130945,e3c2b4c6-d8b7-4b64-8cd0-7ca0e622f9b5
file_name,All CSV files renamed by date/03.14.23.1117.csv,v.1.0/clean-max15.csv,v.1.0/clean-max16.csv,DCEMS Round 2 only triage sessions/Frank M.csv,v.1.0/clean-b0a0d36e-4fb8-43b7-a966-d730e11309...,v.1.0/Clean Marty20.csv
logger_version,1.3,1.0,1.0,1.3,1.0,1.0
tool_applied_type,Needle,Needle,Needle,Needle,Needle,Needle
tool_applied_attachment_point,RChest_Breathe1 (UnityEngine.GameObject),RChest_Breathe1 (UnityEngine.GameObject),RChest_Breathe1 (UnityEngine.GameObject),RChest_Breathe1 (UnityEngine.GameObject),RChest_Breathe1 (UnityEngine.GameObject),RChest_Breathe1 (UnityEngine.GameObject)
tool_applied_tool_location,cath_needle (UnityEngine.GameObject),cath_needle (UnityEngine.GameObject),cath_needle (UnityEngine.GameObject),cath_needle (UnityEngine.GameObject),cath_needle (UnityEngine.GameObject),cath_needle (UnityEngine.GameObject)
tool_applied_data,right_chest,right_chest,right_chest,right_chest,right_chest,right_chest


In [45]:

match_series = (frvrs_logs_df.tool_applied_sender == 'Needle_Decomp(Clone) (UnityEngine.GameObject)') & frvrs_logs_df.tool_applied_patient_id.isnull()
for (session_uuid, scene_index), tool_applied_df in frvrs_logs_df[match_series].groupby(fu.scene_groupby_columns):
    
    # Get the elapsed time history of this scene
    tool_applied_mask_series = True
    for cn in fu.scene_groupby_columns: tool_applied_mask_series &= (frvrs_logs_df[cn] == eval(cn))
    action_tick = tool_applied_df.action_tick.min()
    tool_applied_mask_series &= (frvrs_logs_df.action_tick == action_tick)
    scene_df = frvrs_logs_df[tool_applied_mask_series]
    # display(scene_df.sample(min(6, scene_df.shape[0])).dropna(axis='columns', how='all').T); break

    # Set patient ID to the first valid instance
    patient_id = None
    srs = scene_df.patient_id
    first_valid_index = srs.first_valid_index()
    if first_valid_index is None:
        
        # Get the entire history of this scene
        mask_series = True
        for cn in fu.scene_groupby_columns: mask_series &= (frvrs_logs_df[cn] == eval(cn))
        scene_df = frvrs_logs_df[mask_series]
        
        # Get the index of the row where the action_tick equals the given action_tick
        mask_series = (scene_df.action_tick <= action_tick)
        df = scene_df[mask_series].sort_values('action_tick', ascending=False)
        
        # Iterate backwards through the scene data frame from the row_index until a non-null value is found in the patient ID column
        for i in df.index:
            if not pd.isna(scene_df.loc[i, 'patient_id']):
                patient_id = scene_df.loc[i, 'patient_id']
                break
        
    else: patient_id = srs[first_valid_index]
    if patient_id is not None:
        frvrs_logs_df.loc[tool_applied_mask_series, 'patient_id'] = patient_id
        tool_applied_mask_series &= (frvrs_logs_df.action_type == 'TOOL_APPLIED')
        frvrs_logs_df.loc[tool_applied_mask_series, 'tool_applied_patient_id'] = patient_id

In [46]:

match_series = (frvrs_logs_df.tool_applied_sender == 'Needle_Decomp(Clone) (UnityEngine.GameObject)') & frvrs_logs_df.tool_applied_patient_id.isnull()
df = frvrs_logs_df[match_series]
print(df.shape) # (25, 113)
df.sample(min(6, df.shape[0])).dropna(axis='columns', how='all').T

(0, 113)


""


In [47]:

nu.store_objects(frvrs_logs_df=frvrs_logs_df)
print(frvrs_logs_df.shape) # (842663, 111)

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\frvrs_logs_df.pkl
(829277, 113)
